In [1]:
import sys
import os
# Get the parent directory path
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

Load the images names

In [2]:
images = []
dataDir = "../images1/"
for img in os.listdir(dataDir):

    #join the path with the names of the images
    dataDirTemp = os.path.join(dataDir, str(img))
    images.append(dataDirTemp)
print("Images to be processed: ", images)
#Debug
#images = images[7:8]

Images to be processed:  ['../images1/G000_IMG062.jpg', '../images1/G000_IMG087.jpg', '../images1/G000_IMG102.jpg', '../images1/G006_IMG048.jpg', '../images1/G006_IMG086.jpg', '../images1/G006_IMG119.jpg', '../images1/G019_IMG082.jpg', '../images1/G028_IMG015.jpg', '../images1/G028_IMG062.jpg', '../images1/G028_IMG098.jpg', '../images1/G028_IMG101.jpg', '../images1/G033_IMG043.jpg', '../images1/G033_IMG075.jpg', '../images1/G033_IMG088.jpg', '../images1/G033_IMG101.jpg', '../images1/G038_IMG074.jpg', '../images1/G038_IMG088.jpg', '../images1/G038_IMG103.jpg', '../images1/G038_IMG105.jpg', '../images1/G041_IMG042.jpg', '../images1/G041_IMG048.jpg', '../images1/G041_IMG088.jpg', '../images1/G041_IMG098.jpg', '../images1/G047_IMG053.jpg', '../images1/G047_IMG068.jpg', '../images1/G047_IMG102.jpg', '../images1/G047_IMG107.jpg', '../images1/G056_IMG017.jpg', '../images1/G056_IMG077.jpg', '../images1/G056_IMG097.jpg', '../images1/G058_IMG044.jpg', '../images1/G058_IMG074.jpg', '../images1/G0

In [3]:
import cv2
import numpy as np
import copy

def findHorse(img_original, x_offset=0, y_offset=0, width_box=30, height_box=30,debug=False):
    """
    Draws boxes at the four corners of an image.
    
    Parameters:
    - img: Input image (BGR format)
    - x_offset, y_offset: Offset for corner positions
    - width_box, height_box: Size of the corner boxes
    """
    img = img_original.copy()
    # Convert to grayscale (though we'll draw on the original color image)
    black_and_white = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    height, width = black_and_white.shape[:2]
    
    # Define the four corners with optional offsets
    corners = [
        (x_offset, height - 1 - height_box - y_offset),            # Bottom-left
        (y_offset, x_offset),                          # Top-left
        (width - 1 - height_box - y_offset, height - 1 - width_box - x_offset), # Bottom-right
        (width - 1 - width_box - x_offset , y_offset)             # Top-right
    ]
    convert = ["top_left",  "bottom_left","top_right", "bottom_right"]
    # Create a list to hold all box contours
    all_contours = []
    intensities = []
    count = 1
    for (x, y) in corners:
        if count%2 ==0:
            temp = width_box
            width_box = height_box
            height_box = temp
        count += 1
        # Define a rectangle (box) around the corner point
        box = np.array([
            [x, y],                          # Top-left of the box
            [x + width_box, y],               # Top-right of the box
            [x + width_box, y + height_box],  # Bottom-right of the box
            [x, y + height_box]               # Bottom-left of the box
        ], dtype=np.int32)
        
        all_contours.append(box)
        mask = np.zeros_like(black_and_white, dtype=np.uint8)
        cv2.drawContours(mask, [box], -1, 255, -1)  # Fill the contour (thickness=-1)
        mean_intensity = cv2.mean(black_and_white, mask=mask)[0]
        intensities.append(mean_intensity)
    index = np.argmin(intensities)

    # Draw all the box contours on the original image
    cv2.drawContours(
        img,
        [all_contours[index]],
        #all_contours,
        contourIdx=-1,          # Draw all contours
        color=(0, 255, 0),      # Green color (BGR)
        thickness=2             # Line thickness
    )
    oriented_board = copy.deepcopy(img)
    # Display results
    rotation_code = 0
    if index == 2:
        rotation_code = 90  
        oriented_board = cv2.rotate(oriented_board, cv2.ROTATE_90_CLOCKWISE)
    elif index == 3:
        rotation_code = 180  
        oriented_board = cv2.rotate(oriented_board, cv2.ROTATE_180)
    elif index == 1:
        rotation_code = 260 
        oriented_board = cv2.rotate(oriented_board, cv2.ROTATE_90_COUNTERCLOCKWISE)
    if(debug):
        cv2.imshow("Image with Corner Boxes", img)
        cv2.imshow("Oriented Board", oriented_board)
        cv2.waitKey(0)
        #cv2.imshow("Grayscale Version", black_and_white)
        #cv2.waitKey(0)
        cv2.destroyAllWindows()
    
    return oriented_board,rotation_code

# Example usage:
# image = cv2.imread("your_image.jpg")
# processed_image = findCorners(image, width_box=40, height_box=40)

# Yolov11M

In [4]:
from ultralytics import YOLO
yoloResolution = 1856
def runYoloOnImages(images):
    results = []
    model = YOLO("best.pt")
    batch_size = 4
    # Calculate number of batches needed (rounding up)
    num_batches = (len(images) + batch_size - 1) // batch_size

    # Split into batches
    batches = [images[i*batch_size : (i+1)*batch_size] 
            for i in range(num_batches)]

    # Then process each batch
    for batch in batches:
        result = model.predict(batch, save=True, imgsz=yoloResolution, conf=0.6,batch=1)
        results.extend(result)
    return results
yoloResults = runYoloOnImages(images)
print(len(yoloResults))


0: 1856x1856 3 white-pawns, 2 white-rooks, 1 white-knight, 1 white-king, 4 black-pawns, 2 black-rooks, 1 black-knight, 1 black-king, 142.0ms
1: 1856x1856 1 white-pawn, 1 white-rook, 1 white-knight, 1 white-king, 3 black-pawns, 1 black-rook, 1 black-knight, 1 black-king, 142.0ms
2: 1856x1856 1 white-pawn, 1 white-rook, 1 white-king, 3 black-pawns, 1 black-rook, 1 black-king, 142.0ms
3: 1856x1856 6 white-pawns, 2 white-rooks, 1 white-knight, 1 white-bishop, 1 white-queen, 1 white-king, 6 black-pawns, 2 black-rooks, 1 black-knight, 1 black-bishop, 1 black-king, 1 black-king, 142.0ms
Speed: 24.4ms preprocess, 142.0ms inference, 20.2ms postprocess per image at shape (1, 3, 1856, 1856)
Results saved to runs\detect\predict33

0: 1856x1856 3 white-pawns, 1 white-rook, 1 white-knight, 1 white-king, 3 black-pawns, 1 black-rook, 1 black-king, 124.7ms
1: 1856x1856 2 white-pawns, 1 white-rook, 1 white-knight, 1 white-king, 1 black-pawn, 1 black-rook, 1 black-king, 124.7ms
2: 1856x1856 4 white-pawn

# Convert yolo bounding boxes to contours

In [5]:
def rect_to_contour(x1, y1, x2, y2):
    # Convert coordinates to integers
    x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
    # Contour must be a numpy array of shape (N, 1, 2)
    contour = np.array([
        [[x1, y1]],  # Top-left
        [[x2, y1]],  # Top-right
        [[x2, y2]],  # Bottom-right
        [[x1, y2]]   # Bottom-left
    ], dtype=np.int32)
    return contour
YoloContours = []
YoloBBids =  []
for i,img in enumerate(images):
    yoloResult = yoloResults[i]
    if len(yoloResult.boxes) == 0:
        print("No boxes detected in image: ", img)
        continue
    img_original = cv2.imread(img)
    #img_original = cv2.resize(img_original, (yoloResolution, yoloResolution))
    img_height, img_width = img_original.shape[:2]
    print("Processing image: ", img, " with shape: ", img_original.shape)
    contours = []
    BBids = []
    for box in yoloResult.boxes:
        xyxy = box.xyxy[0].cpu().numpy()
        #print("Box coordinates: ", xyxy)
        contour = rect_to_contour(xyxy[0], xyxy[1], xyxy[2], xyxy[3])
        #print("Contour: ", contour)
        contours.append(contour)
        #print(int(box.cls.cpu()))
        BBids.append(int(box.cls.cpu()))
    YoloContours.append(contours)
    YoloBBids.append(BBids)
    #cv2.drawContours(img_original, contours, -1, (0, 255, 0), 4)
    #img_original = cv2.resize(img_original, (720, 720), img_original)
    #cv2.imshow("YOLO Boxes", img_original)
    #cv2.waitKey(0)
    #cv2.destroyAllWindows()




Processing image:  ../images1/G000_IMG062.jpg  with shape:  (3072, 3072, 3)
Processing image:  ../images1/G000_IMG087.jpg  with shape:  (3072, 3072, 3)
Processing image:  ../images1/G000_IMG102.jpg  with shape:  (3072, 3072, 3)
Processing image:  ../images1/G006_IMG048.jpg  with shape:  (3072, 3072, 3)
Processing image:  ../images1/G006_IMG086.jpg  with shape:  (3072, 3072, 3)
Processing image:  ../images1/G006_IMG119.jpg  with shape:  (3072, 3072, 3)
Processing image:  ../images1/G019_IMG082.jpg  with shape:  (3024, 3024, 3)
Processing image:  ../images1/G028_IMG015.jpg  with shape:  (3024, 3024, 3)
Processing image:  ../images1/G028_IMG062.jpg  with shape:  (3024, 3024, 3)
Processing image:  ../images1/G028_IMG098.jpg  with shape:  (3024, 3024, 3)
Processing image:  ../images1/G028_IMG101.jpg  with shape:  (3024, 3024, 3)
Processing image:  ../images1/G033_IMG043.jpg  with shape:  (3024, 3024, 3)
Processing image:  ../images1/G033_IMG075.jpg  with shape:  (3024, 3024, 3)
Processing i

In [6]:
def transform_contours_to_warped_space(contours, M, fx=1, fy=1):
    """
    Transforms contours from original image space to warped space.
    
    Args:
        contours: List of contours (each as numpy array with shape (N,1,2))
        M: Perspective transformation matrix (3x3)
        fx: x-axis scale factor from warping (default=1)
        fy: y-axis scale factor from warping (default=1)
    
    Returns:
        List of transformed contours in warped image space
    """
    warped_contours = []
    
    for cnt in contours:
        # Convert contour to (N,2) format and float32
        cnt_pts = cnt.reshape(-1, 2).astype(np.float32)
        
        # Apply perspective transform
        warped_pts = cv2.perspectiveTransform(cnt_pts.reshape(1, -1, 2), M)[0]
        
        # Apply scaling if needed
        if fx != 1 or fy != 1:
            warped_pts[:, 0] *= fx
            warped_pts[:, 1] *= fy
        
        # Reshape back to (N,1,2) and convert to integers
        warped_cnt = warped_pts.reshape(-1, 1, 2).astype(np.int32)
        warped_contours.append(warped_cnt)
    
    return warped_contours

# IOU WITH YOLO

In [7]:
from chessboardPieces import drawSquares
def compute_iou(contour1, contour2):
    """Compute Intersection over Union (IoU) between two contours."""
    # Create blank images
    img1 = np.zeros((1000, 1000), dtype=np.uint8)  # Adjust size if needed
    img2 = np.zeros((1000, 1000), dtype=np.uint8)
    
    # Draw contours
    cv2.drawContours(img1, [contour1], -1, 255, -1)  # Filled contour
    cv2.drawContours(img2, [contour2], -1, 255, -1)
    
    # Compute intersection and union
    intersection = np.logical_and(img1, img2)
    union = np.logical_or(img1, img2)
    
    iou = np.sum(intersection) / np.sum(union)
    return iou

def getBestSquareByIou(contour, square_box,normalizedBoard):
    row_idx = -1
    bestIou = 0
    bestPosition = None
    normalizedBoard = copy.deepcopy(normalizedBoard)
    for row in square_box:
        row_idx += 1
        column_idx = -1
        for square in row:
            column_idx += 1
            if square is None:
                #print("Square is None")
                continue
            """print("Debug Contour :",square)
            cv2.drawContours(normalizedBoard, [square], -1, (0, 255, 0), 4)
            cv2.imshow("YOLO Boxes", normalizedBoard)
            cv2.waitKey(0)
            cv2.destroyAllWindows()"""
            iou = compute_iou(contour, square)
            if(iou > 0):
                bestIou = iou
                bestPosition = (row_idx, column_idx)
    return bestPosition, bestIou

def getBoardState(YoloBBids,warped_yoloBB, square_box, rotation,normalizedBoard):
    result = [[12 for _ in range(8)] for _ in range(8)]
    for id,contour in zip(YoloBBids,warped_yoloBB):
        """print("Debug Contour :",contour)
        cv2.drawContours(normalizedBoard, [contour], -1, (0, 255, 0), 4)
        cv2.imshow("YOLO Boxes", normalizedBoard)
        cv2.waitKey(0)
        cv2.destroyAllWindows()"""

        square_box_tmp = copy.deepcopy(square_box)
        #square_box_tmp1 = copy.deepcopy(square_box)
        #drawSquares(square_box_tmp1,copy.deepcopy(normalizedBoard))
        point,iou =getBestSquareByIou(contour, square_box_tmp,normalizedBoard)
        if point is None:
            print("Continue")
            print("Debug Contour :",contour)
            cv2.drawContours(normalizedBoard, [contour], -1, (0, 255, 0), 4)
            cv2.imshow("YOLO Boxes", normalizedBoard)
            cv2.waitKey(0)
            cv2.destroyAllWindows()
            continue
        if (result[point[0]][point[1]]!=12):
            print("WARNING OVERRIDED")
        result[point[0]][point[1]] = id
    return result

In [8]:
import cv2
import numpy as np

def drawBoard(matrix):
    pathPieces = "../assets/resources/pieces"
    pathBoard = "../assets/resources/board.png"
    pieces = ["white-pawn.png","white-rook.png","white-knight.png","white-bishop.png","white-queen.png","white-king.png","black-pawn.png","black-rook.png","black-knight.png",
             "black-bishop.png","black-queen.png","black-king.png"]
    
    # Load the board image
    board = cv2.imread(pathBoard)
    if board is None:
        raise FileNotFoundError(f"Board image not found at {pathBoard}")
    
    height, width = board.shape[:2]
    square_size = height // 8  # Assuming a standard 8x8 chess board
    
    # Load and resize all piece images
    piece_images = []
    for piece in pieces:
        piece_path = f"{pathPieces}/{piece}"
        img = cv2.imread(piece_path, cv2.IMREAD_UNCHANGED)
        if img is None:
            raise FileNotFoundError(f"Piece image not found at {piece_path}")
        # Resize piece to fit the square
        img = cv2.resize(img, (square_size, square_size))
        piece_images.append(img)
    
    # Draw pieces on the board based on the matrix
    for row in range(8):
        for col in range(8):
            piece_index = matrix[row][col]
            if piece_index != 12:  # Assuming -1 or similar means no piece
                # Calculate position
                x = col * square_size
                y = row * square_size
                
                # Get the appropriate piece image
                piece_img = piece_images[piece_index]
                
                # If the image has an alpha channel (transparency)
                if piece_img.shape[2] == 4:
                    # Overlay the piece with transparency handling
                    overlay_img = piece_img[:, :, :3]
                    mask = piece_img[:, :, 3] / 255.0
                    
                    # Get the region of interest
                    roi = board[y:y+square_size, x:x+square_size]
                    
                    # Blend the images
                    for c in range(3):
                        roi[:, :, c] = roi[:, :, c] * (1 - mask) + overlay_img[:, :, c] * mask
                else:
                    # No alpha channel, simple overlay
                    board[y:y+square_size, x:x+square_size] = piece_img
    
    return board

In [ ]:
from task3_2API import detectBoardAndSquares
from chessboardPieces import drawSquares
for index,img in enumerate(images):
    print("Processing image: ", img)
    yoloBB = YoloContours[index]
    yoloBBid = YoloBBids[index]
    # Call the function to detect board and squares
    normalizedBoard, square_box, M, fx, fy = detectBoardAndSquares(img)
    if(normalizedBoard is not None):
        
        orientedBoard,rotation  =findHorse(normalizedBoard,10,5,45,70,debug=False)
        """cv2.imshow("Oriented Board", orientedBoard)
        cv2.waitKey(0)
        cv2.destroyAllWindows()"""
        warped_yoloBB = transform_contours_to_warped_space(yoloBB, M, fy, fx)
        """cv2.drawContours(normalizedBoard, warped_yoloBB, -1, (0, 255, 0), 4)
        normalizedBoard1 = cv2.resize(normalizedBoard, (720, 720), normalizedBoard)
        cv2.imshow("YOLO Boxes", normalizedBoard1)
        cv2.waitKey(0)
        cv2.destroyAllWindows()"""
        square_box2 = copy.deepcopy(square_box)
        #square_box1 = copy.deepcopy(square_box)
        #drawSquares(square_box1,copy.deepcopy(normalizedBoard))
        matrix = getBoardState(yoloBBid,warped_yoloBB, square_box2, rotation,normalizedBoard)
        board = drawBoard(matrix)
        cv2.imshow("Ground Truth ", normalizedBoard)
        cv2.imshow("Result", board)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
# Print column by column
"""
        for col in range(8):
            for row in range(8):
                print(matrix[row][col], end=' ')
            print() """ # New line after each column
                #drawSquares(square_box,normalizedBoard)

Processing image:  ../images1/G000_IMG062.jpg
All squares found
WARNING OVERRIDED
WARNING OVERRIDED
Processing image:  ../images1/G000_IMG087.jpg
All squares found
Processing image:  ../images1/G000_IMG102.jpg
All squares found
Processing image:  ../images1/G006_IMG048.jpg
All squares found
WARNING OVERRIDED
WARNING OVERRIDED
WARNING OVERRIDED
Processing image:  ../images1/G006_IMG086.jpg
All squares found
Processing image:  ../images1/G006_IMG119.jpg
All squares found
Processing image:  ../images1/G019_IMG082.jpg
All squares found
WARNING OVERRIDED
Processing image:  ../images1/G028_IMG015.jpg
Processing image:  ../images1/G028_IMG062.jpg
All squares found
Processing image:  ../images1/G028_IMG098.jpg
All squares found
WARNING OVERRIDED
Processing image:  ../images1/G028_IMG101.jpg
All squares found
Processing image:  ../images1/G033_IMG043.jpg
All squares found
WARNING OVERRIDED
Processing image:  ../images1/G033_IMG075.jpg
All squares found
Processing image:  ../images1/G033_IMG088.